In [1]:
import pandas as pd 
import numpy as np
import re
import os
import argparse
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential,Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import SimpleRNN , Dropout ,LSTM,Conv1D, GlobalMaxPooling1D, MaxPooling1D , SpatialDropout1D,Activation,Dense, Dropout, Activation, Flatten, Input, concatenate , GRU
from keras.layers.embeddings import Embedding
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

In [2]:
nltk.download('stopwords')
nltk.download('punkt')
stop_words = stopwords.words('arabic')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mahmoud\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mahmoud\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [22]:
dataset1 = pd.read_csv('C:\\Users\\Mahmoud\\Desktop\\Book2.csv')
dataset2 = pd.read_csv('C:\\Users\\Mahmoud\\Desktop\\OSACT2020-sharedTask-dev.tsv' , delimiter = '\t' , quoting = 3)
dataset3 = pd.read_csv('C:\\Users\\Mahmoud\\Desktop\\OSACT2020-sharedTask-train.tsv' , delimiter = '\t' , quoting = 3)
dataset4 =dataset2.iloc[:,0:2]
dataset5 =dataset3.iloc[:,0:2]
df_concat = pd.concat([dataset5,dataset4])
for i in range(0,len(df_concat)):
    if df_concat['Class'].iloc[i] == 'normal' or df_concat['Class'].iloc[i] == 'NOT_OFF' or df_concat['Class'].iloc[i] == 'Non-Offensive' or df_concat['Class'].iloc[i] == 0:
        df_concat['Class'].iloc[i] = 'N'
    elif df_concat['Class'].iloc[i] == 'abusive' or df_concat['Class'].iloc[i] == 'hate' or df_concat['Class'].iloc[i] == 'OFF' or df_concat['Class'].iloc[i] == 'Offensive' or df_concat['Class'].iloc[i] == -1 or df_concat['Class'].iloc[i] == -2:
        df_concat['Class'].iloc[i] = 'P'
print(df_concat.shape)

(8000, 2)


In [23]:
X = df_concat.iloc[: , 0].values
y = df_concat.iloc[: , -1].values

In [24]:
labelencoder_y=LabelEncoder()
y = labelencoder_y.fit_transform(y)
for i in range(0,len(y)):
    if y[i]==0 or y[i]==1:
        continue
    else:
        y[i]=0
print(max(y))

1


In [25]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size = 0.2 , random_state = 42)

In [26]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)


def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text


def remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', text)
    return text


def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)


def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
        u"\u2066"
        u"\u2069"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', str(data))

def to_lowercase(text):
    return text.lower()

def remove_eng(text):
    return re.sub(r'[A-Za-z]','',str(text))
    
import string
def remove_punctuation(text):
    """Remove punctuation from list of tokenized words"""
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)


def replace_numbers(text):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    return re.sub(r'\d+', '', text)


def remove_whitespaces(text):
    return text.strip()


def remove_stopwords(words, stop_words):

    return [word for word in words if word not in stop_words]


def stem_words(words):
    """Stem words in text"""
    stemmer = PorterStemmer()
    return [stemmer.stem(word) for word in words]

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
def lemmatize_words(words):
    """Lemmatize words in text"""

    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in words]

def lemmatize_verbs(words):
    """Lemmatize verbs in text"""

    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word, pos='v') for word in words])

def text2words(text):
  return word_tokenize(text)

def normalize_text( text):
    text = remove_emojis(text)
    text = remove_eng(text)
    text = remove_punctuation(text)
    text = remove_punctuations(text)
    text = normalize_arabic(text)
    text = remove_diacritics(text)
    text = remove_repeating_char(text)
    text = to_lowercase(text)
    text = replace_numbers(text)
    words = text2words(text)
    words = remove_stopwords(words, stop_words)
    #words = stem_words(words)# Either stem ovocar lemmatize
    words = lemmatize_words(words)
    words = lemmatize_verbs(words)

    return ''.join(words)

def normalize_corpus(corpus):
  return [normalize_text(t) for t in corpus]

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mahmoud\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [27]:
X_train = normalize_corpus(X_train)
X_test = normalize_corpus(X_test)

In [28]:
tok = Tokenizer()
tok.fit_on_texts(X_train + X_test)
vocab_size = len(tok.word_index) + 1
encoded_docs_train = tok.texts_to_sequences(X_train)
encoded_docs_test = tok.texts_to_sequences(X_test)
#encoded_docs_check = tok.texts_to_sequences(nor_check)

max_length = 300
padded_docs_train = pad_sequences(encoded_docs_train, maxlen=max_length, padding='post')
padded_docs_test = pad_sequences(encoded_docs_test, maxlen=max_length, padding='post')
#padded_docs_check = pad_sequences(encoded_docs_check, maxlen=max_length, padding='post')
print(padded_docs_train)
print(np.shape(padded_docs_train))

[[   78  6418   222 ...     0     0     0]
 [  117   901 10980 ...     0     0     0]
 [ 2796  1952   769 ...     0     0     0]
 ...
 [28485 28486  1806 ...     0     0     0]
 [  656   460  2802 ...     0     0     0]
 [    5   406   882 ...     0     0     0]]
(6400, 300)


In [33]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(Dropout(0.2))
model.add(LSTM(100 , dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1,activation='sigmoid'))

In [34]:
import tensorflow as tf
import keras
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [35]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 300, 100)          3290100   
                                                                 
 dropout_5 (Dropout)         (None, 300, 100)          0         
                                                                 
 lstm_4 (LSTM)               (None, 100)               80400     
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 3,370,601
Trainable params: 3,370,601
Non-trainable params: 0
_________________________________________________________________


In [36]:
history = model.fit(padded_docs_train, y_train , validation_split = 0.2 , epochs=10 , batch_size = 1024)

Epoch 1/10
5/5 [==============================] - 51s 10s/step - loss: 0.6372 - accuracy: 0.8029 - val_loss: 0.5712 - val_accuracy: 0.7812
Epoch 2/10
5/5 [==============================] - 48s 10s/step - loss: 0.5198 - accuracy: 0.8094 - val_loss: 0.5737 - val_accuracy: 0.7812
Epoch 3/10
5/5 [==============================] - 48s 10s/step - loss: 0.4939 - accuracy: 0.8094 - val_loss: 0.5259 - val_accuracy: 0.7812
Epoch 4/10
5/5 [==============================] - 49s 10s/step - loss: 0.4942 - accuracy: 0.8094 - val_loss: 0.5259 - val_accuracy: 0.7812
Epoch 5/10
5/5 [==============================] - 48s 10s/step - loss: 0.4895 - accuracy: 0.8094 - val_loss: 0.5297 - val_accuracy: 0.7812
Epoch 6/10
5/5 [==============================] - 57s 12s/step - loss: 0.4880 - accuracy: 0.8094 - val_loss: 0.5342 - val_accuracy: 0.7812
Epoch 7/10
5/5 [==============================] - 53s 10s/step - loss: 0.4887 - accuracy: 0.8094 - val_loss: 0.5277 - val_accuracy: 0.7812
Epoch 8/10
5/5 [===========

In [15]:
loss, accuracy = model.evaluate(padded_docs_test, y_test)
print('Accuracy: %f' % (accuracy*100),'  ',loss)

50/50 [==============================] - 2s 45ms/step - loss: 0.5125 - accuracy: 0.7919
Accuracy: 79.187500    0.5124989151954651


In [16]:
model.save("try_model_LSTM.h5")

In [17]:
import pickle

# to save the fitted tokenizer
with open('tokenizer_LSTM.pickle', 'wb') as handle:
    pickle.dump(tok, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder_y_LSTM.pickle', 'wb') as ecn_file:
    pickle.dump(labelencoder_y, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)